In [1]:
# Yonghwan Kim Assignment3 Task3
!pip install pyspark

In [2]:
import sys
import itertools
from pyspark import SparkContext, SparkConf

In [3]:
# create Spark context with necessary configuration
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('Task3')
sc = SparkContext(conf=conf)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Give **.txt FILE PATHS** here
# You can use any random input file of your choice
file3Dir = '/content/drive/MyDrive/Assignment3_cloud/Assignment3_Task3Data.txt'
fileSaveDir = '/content/drive/MyDrive/Assignment3_cloud/Task3Output'
# USE THIS FILE as input FOR ALL BELOW tasks
# Change filepath to HDFS path when executing in the cluster

In [6]:
# Iterate through the data once and fill direct_friends and friend_recommendations
def friendMapping(i):
  direct_friends = list()
  friend_recommendations = list()

  userId, friends = i[0], i[1]
  # direct_friends
  for friend in friends:
    if userId > friend:
      direct_friends.append(((friend, userId), 0))
    else:
      direct_friends.append(((userId, friend), 0))

  # friend_recommendations  
  for friendComb in itertools.combinations(friends, 2):
    if friendComb[0] > friendComb[1]:
      friend_recommendations.append(((friendComb[1], friendComb[0]), 1))
    else:
      friend_recommendations.append(((friendComb[0], friendComb[1]), 1))
  return direct_friends + friend_recommendations

# recover the string to RDD
# take input file from TF_index directory
fileRDD = sc.textFile(file3Dir)
formatRDD1 = fileRDD.map(lambda i: i.split("\t"))
formatRDD2 = formatRDD1.map(lambda i: (i[0], i[1].split(",")))

# Stores directly connected friends and user recommendations
friendMappingRDD = formatRDD2.flatMap(friendMapping)

In [7]:
# group the rdd by key
groupRDD = friendMappingRDD.groupByKey()
# remove direct friends from friend_recommendations
filterRDD = groupRDD.filter(lambda i: 0 not in i[1])
# sum up weights
sumRDD = filterRDD.map(lambda i: (i[0], sum(i[1])))
# recommendation
recommdRDD = sumRDD.flatMap(lambda i: [(i[0][0], (i[0][1], i[1])), (i[0][1], (i[0][0], i[1]))])
# group the rdd by key
groupRecommdRDD = recommdRDD.groupByKey()
# resolve the output in list format
resultRDD = groupRecommdRDD.map(lambda i: (i[0], list(i[1])))

In [8]:
# Sort recommendations based on number of mutual friends and user IDs

# sort value by number of mutual friends in decreasing order
def sortRecommendation(result):
  r1 = sorted(result[1], key=lambda item: item[0]) # sort in increasing order first
  r2 = sorted(r1, key=lambda item: (-item[1], item[0])) # sort in decreasing order for friend recommedations
  return [result[0], r2]

# sorting
finalRDD1 = resultRDD.map(sortRecommendation)
finalRDD2 = finalRDD1.map(lambda i: [i[0], i[1][:10]])
finalRDD3 = finalRDD2.map(lambda i: (i[0], [r[0] for r in i[1]]))

# output sample list
outputRecomUserIDList = [924, 1153, 6712, 8213, 12835, 23478]
# broadcast the list
broadcastIDList = sc.broadcast(outputRecomUserIDList)
finalRDD4 = finalRDD3.filter(lambda i: int(i[0]) in broadcastIDList.value)
# format the output
stringRDD = finalRDD4.map(lambda l: l[0] + ": " + ','.join(e for e in l[1]))
stringRDD.saveAsTextFile(fileSaveDir)